## Grid backtest parameter search example

In [1]:
import pandas as pd
import numpy as np
from simple.chart import interactTable, chartProfit
from simple.funcs import vwap
from simple.geneopt import GridOpt
from simple.pretty import pp
from multiprocessing import current_process
from simple.backtest import getProfitDict, npBacktestLimit
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:,.2f}'.format

In [2]:
T = np.load('data/tick.npz')['BTCUSDT'].view(np.recarray)
len(T)

889360

In [3]:
T

rec.array([('2021-06-01T00:00:00.799000', 37265. ,  1.000e+00),
           ('2021-06-01T00:00:00.832000', 37264.5, -1.000e+00),
           ('2021-06-01T00:00:01.098000', 37264.5, -3.700e+02), ...,
           ('2021-06-04T23:59:52.505000', 36849.5, -1.000e+02),
           ('2021-06-04T23:59:52.505000', 36849.5, -2.824e+03),
           ('2021-06-04T23:59:56.400000', 36840.5,  1.200e+02)],
          dtype=[('DateTime', '<M8[us]'), ('Price', '<f8'), ('Size', '<f8')])

In [4]:
def model(Period: int = (1000, 6000, 500), StdDev: float = (2, 3, 0.1)):
    if Period == 0 or StdDev == 0: return {}
    Tick = T.Price
    Center = vwap(T, Period)
    _std = pd.Series(Tick).rolling(Period).std().bfill().values
    qA = Center + _std * StdDev
    qB = Center - _std * StdDev

    # returns profit metrics during grid search or timeseries data during plot
    _trades = npBacktestLimit(T, qA, qB)
    return getProfitDict(_trades) if current_process().daemon else {**locals(), **chartProfit(_trades)}

In [5]:
# Genetic optimizer
G = GridOpt(model)
G.fullSearch()

  0%|          | 0/110 [00:00<?, ?it/s]

6000

In [9]:
# Grid results
P = pd.DataFrame(G.log, columns=G.log_columns).sort_values('Profit', ascending=False)
P

Period  StdDev     Profit  Count  PRatio  AvgMid     RawPnL      Fee  \
89    5000    2.90   1,463.45     33    0.58   55.52   1,832.13   368.68   
79    4500    2.90     779.54     35    0.57   33.53   1,173.43   393.89   
97    5500    2.70    -403.35     43    0.51    1.79      77.04   480.39   
49    3000    2.90    -403.55     44    0.45    2.05      90.08   493.63   
88    5000    2.80    -659.96     37    0.51   -6.65    -245.92   414.03   
..     ...     ...        ...    ...     ...     ...        ...      ...   
30    2500    2.00 -11,374.76    188    0.50  -49.35  -9,277.87 2,096.89   
3     1000    2.30 -11,563.57    336    0.51  -23.23  -7,805.64 3,757.93   
2     1000    2.20 -12,075.71    390    0.51  -19.78  -7,715.43 4,360.27   
1     1000    2.10 -13,249.97    435    0.51  -19.28  -8,388.62 4,861.35   
0     1000    2.00 -16,035.24    469    0.51  -23.02 -10,796.45 5,238.79   

       MidPnL  Sharpe  
89   1,832.13    2.56  
79   1,173.43    1.26  
97      77.04   -0.84  
49      90.08   -0.75  
88    -245.92   -1.29  
..        ...     ...  
30  -9,277.87  -48.09  
3   -7,805.64  -72.18  
2   -7,715.43  -80.44  
1   -8,388.62  -93.18  
0  -10,796.45 -117.23  

[110 rows x 10 columns]

In [11]:
p = P.pivot(columns='Period', index='StdDev', values='Profit')
pp(p)

In [12]:
# results browser
interactTable(model, P, height=500)